# Hybrid Deep Learning Model

## SOMs with ANN for fraud Prediction.

## Credit Card Fraud Detection and Prediction

## SOM

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from minisom import MiniSom
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from pylab import bone, pcolor, colorbar, plot, show

### Getting the Dataset

In [2]:
dataset = pd.read_csv("dataset/creditcard.csv")

In [3]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

## Data Preprocessing

### Feature Scaling

In [4]:
sc = MinMaxScaler(feature_range=(0, 1))
X = sc.fit_transform(X)

Initialize the model

In [5]:
som = MiniSom(x=10,y=10,input_len=X.shape[1],sigma=1.0,learning_rate=0.5)

Train the model

In [6]:
som.random_weights_init(X)
som.train_random(data = X, num_iteration = 100)

## Visualize the results

In [ ]:
bone()
pcolor(som.distance_map().T)
colorbar()
markers = ['o','s']
colors = ['r','g']
for i, x in enumerate(X):
    w = som.winner(x)
    plot(w[0] + 0.5, w[1] + 0.5, markers[y[i]],markeredgecolor=colors[y[i]],markerfacecolor=None,markersize=10,markeredgewidth=2)

show()

## Finding the Frauds

In [10]:
mappings = som.win_map(X)
frauds = mappings[(7,5)]
frauds = sc.inverse_transform(frauds)

## ANN

Creating the matrix of features

In [8]:
customers = dataset.iloc[:,1:-1].values
is_fraud = np.zeros(len(dataset))

In [11]:
for i in range(len(dataset)):
    if dataset.iloc[i, 0] in frauds:
        is_fraud[i] = 1

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
customers = sc.fit_transform(customers)

In [13]:
ANN = Sequential()
ANN.add(Dense(units=6, kernel_initializer = 'uniform',activation='relu',input_dim=customers.shape[1]))
ANN.add(Dense(units=1, kernel_initializer = 'uniform',activation='sigmoid'))

In [14]:
ANN.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [17]:
ANN.fit(customers, is_fraud, epochs = 3, verbose=2)

Epoch 1/3


8901/8901 - 42s - loss: 0.0603 - accuracy: 0.9862 - 42s/epoch - 5ms/step


Epoch 2/3


8901/8901 - 42s - loss: 0.0597 - accuracy: 0.9863 - 42s/epoch - 5ms/step


Epoch 3/3


8901/8901 - 42s - loss: 0.0594 - accuracy: 0.9865 - 42s/epoch - 5ms/step


In [20]:
y_pred = ANN.predict(customers,verbose=2)
y_pred = np.concatenate((dataset.iloc[:,0:1].values, y_pred), axis = 1)

8901/8901 - 6s - 6s/epoch - 715us/step


In [22]:
y_pred = y_pred[y_pred[:, 1].argsort()]

In [23]:
print(y_pred)

[[1.66198000e+05 3.99217982e-32]
 [5.57090000e+04 3.32608455e-11]
 [1.32779000e+05 2.10648471e-10]
 ...
 [5.48590000e+04 5.90781093e-01]
 [7.77300000e+04 5.91814101e-01]
 [6.63830000e+04 6.00196958e-01]]
